In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
from urllib.parse import urljoin
import pandas as pd
from datetime import datetime

In [2]:
import champ_placement as chp

<a id ="top"></a>

### Important functions in the champ_placement class
 - [Year_Link_finder](#years)
 - [month_soup](#month)
 - [recent_champ](#recent_champ)
 - [find_classes](#find_classes)

---
<a id=years></a>

In [3]:
def year_link_finder(min_year=None, max_year=None):
    base_link = "https://www.agilityplaza.com/results/"
    
    if min_year ==None:
        min_year = 2004
    if max_year == None:
        max_year = int(datetime.now().date().strftime("%Y"))

    years = np.arange(min_year, max_year+1,1).astype(str)
    year_link = {}
    for year in years:
        year_link[year] = base_link + year

    return year_link

def current_year_link():
        current_year = str(datetime.now().date().strftime("%Y"))
        base_link = "https://www.agilityplaza.com/results/"
        year_link = base_link+current_year
        return year_link
        
print(year_link_finder(2024))
print(current_year_link())

{'2024': 'https://www.agilityplaza.com/results/2024'}
https://www.agilityplaza.com/results/2024


---
<a id="month"></a>

## Month_soup
this returns the html soup of the current year and the month specified by the `months_ago` argument.


[back to the top](#top)

In [4]:
champ_placement_instance = chp.champ_placement()

def month_soup(self, months_ago=0, return_month=False):
    """
    Extracts a portion of HTML soup corresponding to a specific month's data from agility plaza.

    Args:
        self: Instance of the class containing the method.
        months_ago (int, optional): An integer indicating how many months ago the function should extract data for. Default is 0, representing the current month.
        return_month (bool, optional): A boolean indicating whether to return the month name along with the soup. Default is False.

    Returns:
        list or tuple: If return_month is False, returns a list of HTML elements between the selected month's data. If return_month is True, returns a tuple containing a list of HTML elements and the name of the month.

    Description:
        This function sends an HTTP GET request to the URL specified by self.current_year_link, parses the response content using BeautifulSoup, and locates all <thead> elements within the HTML soup. It selects the <thead> element corresponding to the month specified by months_ago (default is 0 for the current month). It retrieves all HTML elements between the selected <thead> element and the next <thead> element. If return_month is set to True, the function extracts the month name from the selected <thead> element. Finally, it returns either the extracted HTML elements or both the HTML elements and the month name, depending on the value of return_month.
    """
    # Send an HTTP GET request to the URL
    url = self.current_year_link
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all <thead> elements
    theads = soup.find_all("thead")
    
    # Get the first <thead>
    first_thead = theads[months_ago]

    # Find the elements between the first and second <thead>
    elements_between = []
    current_element = first_thead.find_next_sibling()
    while current_element and current_element.name != 'thead':
        elements_between.append(current_element)
        current_element = current_element.find_next_sibling()

    if return_month:
        month = first_thead.text.strip().split(" ")[0]
        return elements_between, month
    else:
        return elements_between


print(month_soup(champ_placement_instance)[:5])


[<tr>
<th>Dates</th>
<th>Name</th>
</tr>, <tr class="clickable-row organization6" data-href="/competition/1335471852/results">
<td>Sat 16</td>
<td>Go Go Hoopers</td>
</tr>, <tr class="clickable-row organization6" data-href="/competition/1459703690/results">
<td>Sat 16</td>
<td>Super Hoopers</td>
</tr>, <tr class="clickable-row organization1" data-href="/competition/1507416262/results">
<td>Sat 16</td>
<td>Kelluki Agility</td>
</tr>, <tr class="clickable-row organization200" data-href="/competition/1825988804/results">
<td>Sat 16 - Sun 17</td>
<td>Team GB Tryout Weekend 2</td>
</tr>]


<a id = "recent_champ"></a>

## recent_champ
finds the link to the show with the most recent championship in it.
There is also `champ_this_year` which should find all champs in the year but it doesn't work - this is mainly here for a later project that might need this feature


[back to the top](#top)

In [5]:
champ_placement_instance = chp.champ_placement()

def recent_champ(self, months_ago=0, print_statement=True):
    """
    Finds the most recent championship competition within a specified range of months.

    Args:
        self: Instance of the class containing the method.
        months_ago (int, optional): Number of months ago to start searching for championships. Default is 0, representing the current month.
        print_statement (bool, optional): Whether to print search progress and results. Default is True.

    Returns:
        tuple or None: A tuple containing the link and name of the most recent championship competition if found, or None if no championship is found within the specified range.

    Description:
        This function searches for the most recent championship competition within a specified range of months. It starts the search from the current month (or a specified number of months ago) and goes back up to 12 months. For each month, it retrieves the HTML soup corresponding to the competition data and checks if any competition contains the word "Championship" in its name. If a championship is found, it returns a tuple containing the link and name of the championship competition. If no championship is found within the specified range, it returns None. The function optionally prints search progress and results based on the value of the print_statement parameter.
    """
    max_months = 12  # Maximum number of months to go back
    for i in range(months_ago, max_months + 1):
        month_soup = self.month_soup(i)
        Name = None
        link = None

        for j in range(1, len(month_soup)):
            td_element = month_soup[j].find_all('td')[-1]

            if "Championship" in td_element.text:
                Name = td_element.text
                link =  self.base_link[:-9] + month_soup[j].get('data-href')
                if print_statement ==True:
                    print(f"Championship found in {td_element.text}, link {link}")
                    
                return link, Name  # Exit the function once Championship is found
                break

        if Name is None:
            if print_statement==True:
                print(f"No competition with 'Championship' in the name was found for {i} months ago. Trying next month.")
            
    if print_statement ==True:
        print("No competition with 'Championship' in the name was found in the last", max_months, "months.")
    return None

def champ_this_year(self, months_ago=0, print_statement=True):
    '''this almost works but there is an index error in the month_soup with the indexing from max_months. Need to fix month_soup funciton to use this.'''
    max_months = 12  # Maximum number of months to go back
    championships = []  # List to store found championships

    for i in range(months_ago, max_months + 1):
        month_soup = self.month_soup(i)

        for j in range(1, len(month_soup)):
            td_element = month_soup[j].find_all('td')[-1]

            if "Championship" in td_element.text:
                Name = td_element.text
                link =  self.base_link[:-9] + month_soup[j].get('data-href')
                championships.append((Name, link))  # Append championship name and link to the list
                
                if print_statement:
                    print(f"Championship found in {Name}, link {link}")

    if not championships:
        print("No competition with 'Championship' in the name was found in the last", max_months, "months.")
        
    return championships

# recent_champ(champ_placement_instance)


recent_champ(champ_placement_instance, months_ago = 2, print_statement = False)

('https://www.agilityplaza.com/competition/2052012361/results',
 'Derbyshire Championship Agility Show')

---
<a id = "find_classes"></a>

## find_classes
This function finds the link to all the championship classes in the competition with a 

[back to the top](#top)

In [6]:
champ_placement_instance = chp.champ_placement()

import pandas as pd

def find_classes(self, months_ago=0, print_statement=False):
    """
    Extracts information about championship classes from the agility plaza website and organizes it into a pandas DataFrame.

    Args:
        self: Instance of the class containing the method.
        months_ago (int, optional): An integer indicating how many months ago the function should search for championship classes. Default is 0, representing the current month.
        print_statement (bool, optional): A boolean indicating whether to print statements during the execution of the function. Default is False.

    Returns:
        pandas.DataFrame: A DataFrame containing information about championship classes, including the class name, link, and height.

    Description:
        This function first retrieves the link and name of the most recent championship competition by calling the 'recent_champ' method. It then sends an HTTP GET request to the retrieved link, parses the response content using BeautifulSoup, and locates all 'card-block' elements within the HTML soup. It iterates through each 'card-block' element to find the championship classes. For each class found, it extracts the class name and link and appends them to a list. After gathering all class information, it creates a pandas DataFrame with columns for the class name, link, and height. The height is derived from the second word in the class name. The function sets the DataFrame index to be composed of the first two words extracted from the class name. Finally, it returns the DataFrame containing the championship class information.
    """
    show_link, show_name = self.recent_champ(months_ago, print_statement)
    
    response = requests.get(show_link)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Finding the day and classes in that day
    div_elements = soup.find_all("div", class_="card-block")
    
    class_data = []
    for day_div in div_elements:
        for a in day_div.find_all('a'):
            if "Championship Jumping" in a.text:
                name = a.text
                link = "agilityplaza.com" + a.get('href')
                class_data.append((name, link))
            elif "Championship Agility" in a.text:
                name = a.text
                link = "agilityplaza.com" + a.get('href')
                class_data.append((name, link))
    
    # Create a pandas DataFrame
    df = pd.DataFrame(class_data, columns=['Class Name', 'Link'])
    df['class number'] = df['Class Name'].apply(lambda x: ' '.join(x.split()[:1]))
    df.set_index('class number', inplace=True)
    df['Height'] = df['Class Name'].apply(lambda x: x.split()[1] if len(x.split()) >= 2 else None)
    return df

array = find_classes(champ_placement_instance, months_ago = 2, print_statement = False)
array

,Class Name,Link,Height
class number,,,
1a,1a Lge Championship Agility,agilityplaza.com/agilityClass/1227118621/results,Lge
1b,1b Lge Championship Jumping,agilityplaza.com/agilityClass/1496930129/results,Lge
2a,2a Int Championship Agility,agilityplaza.com/agilityClass/2045207625/results,Int
2b,2b Int Championship Jumping,agilityplaza.com/agilityClass/1468004139/results,Int


### his only works for shows with championship in the name. Lots of shows don't have this -> need to sort this out to get the appropriate show

KCI, Derbyshire, Lune Valley, Thames, ..... do not have this

This is going to be solved using the KC page https://www.thekennelclub.org.uk/events-and-activities/agility/already-competing-in-agility/qualifying-shows-for-the-kennel-club-events/ to find the shows. This will not work with the data collection database idea

In [15]:
heights = ['Lge', 'Int', 'Med', 'Sml']
champ_show_link = "https://www.thekennelclub.org.uk/events-and-activities/agility/already-competing-in-agility/qualifying-shows-for-the-kennel-club-events/"

response = requests.get(champ_show_link)
champ_soup = BeautifulSoup(response.content, 'html.parser')

# print(type(champ_show_link))

soup = champ_soup.find_all("details", class_ = "a-details")

for event in soup:
    print(event.find_all("summary"))



                
                

[<summary class="a-details__summary">Crufts singles - small</summary>]
[<summary class="a-details__summary">Crufts singles - medium </summary>]
[<summary class="a-details__summary">Crufts singles - intermediate</summary>]
[<summary class="a-details__summary">Crufts singles - large</summary>]
[<summary class="a-details__summary">Crufts medium ABC agility stakes</summary>]
[<summary class="a-details__summary">Crufts intermediate novice ABC agility stakes</summary>]
[<summary class="a-details__summary">Crufts large novice ABC agility stakes</summary>]
[<summary class="a-details__summary">Small novice cup</summary>]
[<summary class="a-details__summary">Medium novice cup</summary>]
[<summary class="a-details__summary">Intermediate novice cup</summary>]
[<summary class="a-details__summary">Large novice cup</summary>]
[<summary class="a-details__summary">Championship Classes – small</summary>]
[<summary class="a-details__summary">Championship Classes – medium</summary>]
[<summary class="a-det